In [12]:
import os
import json5
import json
import math
import pandas as pd
import numpy as np
from datasets import load_dataset,Dataset
from collections import Counter

In [22]:
origin_xlsx = '/mnt/users/LLMa/ChatGLM-6B/data/LUNG/origin_chest_ct_report_reformat.csv'
total_xlsx =  '/mnt/users/LLMa/ChatGLM-6B/data/LUNG/chest_0411_2592.xlsx'
percentile = 0.15

In [14]:
total_df = pd.read_excel(total_xlsx,)
train_df = total_df[int(len(total_df)*percentile):]
val_df = total_df[:int(len(total_df)*percentile)]

In [79]:
train_pairs = {"Input":list(train_df['report_evidences']),
         "Output":list(train_df['json_report'])}
train_data = Dataset.from_dict(train_pairs)
print(len(train_data))
# train_data.to_csv('/mnt/users/LLMa/ChatGLM-6B/data/LUNG/train.csv')
train_data.to_json('/mnt/users/LLMa/ChatGLM-6B/data/LUNG/train.json',force_ascii=False)


2203


Creating json from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

1517887

In [80]:
val_pairs = {"Input":list(val_df['report_evidences']),
         "Output":list(val_df['json_report'])}
val_data = Dataset.from_dict(val_pairs)
print(len(val_data))
# val_data.to_csv('/mnt/users/LLMa/ChatGLM-6B/data/LUNG/val.csv')
val_data.to_json('/mnt/users/LLMa/ChatGLM-6B/data/LUNG/val.json',force_ascii=False)

388


Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

263272

## Get report to conclusion

In [23]:
origin_df = pd.read_csv(origin_xlsx)

In [24]:

origin_df['report_evidences'].replace('', np.nan, inplace=True)
origin_df.dropna(subset=['report_evidences'], inplace=True)

origin_df['report_conclusion'].replace('', np.nan, inplace=True)
origin_df.dropna(subset=['report_conclusion'], inplace=True)

origin_df['report_conclusion'].replace('id:0',np.nan,inplace=True)
origin_df.dropna(subset=['report_conclusion'], inplace=True)

In [25]:
origin_df.replace('\s+','',regex=True,inplace=True) 

In [27]:
report_lens=[]
conlusion_lens =[]
for idx,row  in origin_df.iterrows():
    if not pd.isna(row['report_evidences']):
        report_lens.append(len(row['report_evidences']))
    if not pd.isna(row['report_conclusion']):
        conlusion_lens.append(len(row['report_conclusion']))
print(Counter(report_lens).most_common())
print(Counter(conlusion_lens).most_common())
print(np.sum(np.asarray(report_lens)>512))  # 118
print(np.sum(np.asarray(conlusion_lens)>512)) # 7

[(60, 8543), (67, 8371), (50, 7969), (91, 6452), (78, 5667), (68, 5168), (105, 5105), (103, 4825), (113, 4821), (92, 4779), (84, 4758), (89, 4650), (108, 4614), (107, 4596), (104, 4590), (106, 4572), (112, 4553), (101, 4552), (95, 4537), (102, 4526), (90, 4459), (94, 4458), (114, 4458), (120, 4442), (99, 4396), (100, 4387), (87, 4386), (109, 4374), (88, 4356), (115, 4355), (110, 4350), (98, 4332), (75, 4323), (116, 4283), (118, 4282), (86, 4253), (81, 4238), (122, 4231), (121, 4223), (80, 4218), (97, 4212), (85, 4211), (96, 4206), (111, 4196), (117, 4193), (93, 4174), (119, 4132), (79, 4130), (124, 4103), (123, 4073), (125, 3948), (76, 3935), (127, 3930), (126, 3917), (77, 3881), (83, 3874), (130, 3857), (131, 3829), (133, 3817), (129, 3815), (74, 3811), (82, 3773), (128, 3766), (132, 3734), (72, 3717), (58, 3710), (134, 3697), (136, 3608), (139, 3576), (135, 3566), (138, 3564), (73, 3553), (137, 3553), (140, 3505), (70, 3396), (141, 3365), (142, 3285), (71, 3281), (145, 3245), (144, 3

In [28]:
train_part = origin_df[int(len(origin_df)*percentile):]
val_part = origin_df[:int(len(origin_df)*percentile)]

In [29]:
train_data = Dataset.from_dict({"Input":list(train_part['report_evidences']),
         "Output":list(train_part['report_conclusion'])})
print(len(train_data))
# val_data.to_csv('/mnt/users/LLMa/ChatGLM-6B/data/LUNG/val.csv')
train_data.to_json('/mnt/users/LLMa/ChatGLM-6B/data/LUNG/train_conclusion.json',force_ascii=False)

450469


Creating json from Arrow format:   0%|          | 0/451 [00:00<?, ?ba/s]

236560363

In [21]:
val_data = Dataset.from_dict({"Input":list(val_part['report_evidences']),
         "Output":list(val_part['report_conclusion'])})
print(len(val_data))
# val_data.to_csv('/mnt/users/LLMa/ChatGLM-6B/data/LUNG/val.csv')
val_data.to_json('/mnt/users/LLMa/ChatGLM-6B/data/LUNG/val_conclusion.json',force_ascii=False)

28744


Creating json from Arrow format:   0%|          | 0/29 [00:00<?, ?ba/s]

15161866